<a href="https://colab.research.google.com/github/thisismedha/DeepLearning/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework3
*by Medha Singh*

**Answer4:**
PubMed is a database of biomedical literature, including articles and abstracts from scientific journals, books, and conference proceedings. The database includes over 30 million citations, with content covering a range of biomedical topics such as medicine, nursing, dentistry, veterinary medicine, and public health. PubMed is widely used by researchers, clinicians, and other professionals in the biomedical field to find information and stay up to date with the latest research in their field. It is also commonly used as a resource for natural language processing and machine learning research in the biomedical domain.

In [ ]:
# Install packages
!pip list -v | grep torch
!pip uninstall torch torchvision torchaudio
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch_geometric
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

torch                         2.0.0+cu118          /usr/local/lib/python3.9/dist-packages pip
torchaudio                    2.0.1+cu118          /usr/local/lib/python3.9/dist-packages pip
torchdata                     0.6.0                /usr/local/lib/python3.9/dist-packages pip
torchsummary                  1.5.1                /usr/local/lib/python3.9/dist-packages pip
torchtext                     0.15.1               /usr/local/lib/python3.9/dist-packages pip
torchvision                   0.15.1+cu118         /usr/local/lib/python3.9/dist-packages pip
Found existing installation: torch 2.0.0+cu118
Uninstalling torch-2.0.0+cu118:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.9/dist-packages/functorch/*
    /usr/local/lib/python3.9/dist-packages/nvfuser/*
    /usr/local/lib/python3.9/dist-packages/torch-2.0.0+cu118.dist-info/*
    /usr/local/lib/python3.9/dist-packages/torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 28.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=1d1955f394d2b9e93e8a41c5f944ee27a7e3230e76a667ffacf84ecdc98be92c
  Stored in directory: /root/.cache/pip/wheels/cd/7d/6b/17150450b80b4a3656a84330e22709ccd8dc0f8f4773ba4133
Successfully built torch_geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3

In [ ]:
# import libraries
import torch
import torch_geometric
torch.manual_seed(548)
from torch_geometric.datasets import Reddit

In [ ]:
# load dataset
planetoid = torch_geometric.datasets.Planetoid(root='/tmp/PubMed', name='PubMed')
data = planetoid[0]

Processing...
Done!


In [ ]:
data

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [ ]:
data.y

tensor([1, 1, 0,  ..., 2, 0, 2])

In [ ]:
# define GCN
class GCN(torch.nn.Module):
    def __init__(self, dataset):
        super().__init__()
        self.conv1 = torch_geometric.nn.GCNConv(dataset.num_node_features, 64)
        self.conv2 = torch_geometric.nn.GCNConv(64, 64)
        self.conv2 = torch_geometric.nn.GCNConv(64, dataset.num_classes)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.nn.functional.log_softmax(x, dim=1)
        return x

In [ ]:
# train GCN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = GCN(planetoid).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
# clear memory
import torch
torch.cuda.empty_cache()


In [ ]:
for epoch in range(150):
    optimizer.zero_grad()
    out = model(data)
    loss = torch.nn.functional.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7890


The Accuracy of PubMed data set with 150 epochs is **78.9%**

In [ ]:
# GCN for linke prediction
class GCN(torch.nn.Module):
    def __init__(self, dataset):
        super().__init__()
        self.conv1 = torch_geometric.nn.GCNConv(dataset.num_node_features, 16)
        self.conv2 = torch_geometric.nn.GCNConv(16, 12)
        self.linear1 = torch.nn.Linear(24, 24)
        self.linear2 = torch.nn.Linear(24, 1)

    def forward(self, x, train_edge_index, pred_edge_index):
        x = self.conv1(x, train_edge_index)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x, train_edge_index)
        x = torch.nn.functional.relu(x)

        x_positive = torch.cat((x[pred_edge_index[0,:],:], 
                                x[pred_edge_index[1,:],:]), dim=1)
        x_positive = self.linear1(x_positive)
        x_positive = torch.nn.functional.relu(x_positive)
        x_positive = self.linear2(x_positive)
        x_positive = torch.sigmoid(x_positive)

        x_negative = torch.cat((x[torch.randint(high = x.shape[0], size = (pred_edge_index.shape[1],)),:],
                                x[pred_edge_index[1,:],:]), dim=1)
        x_negative = self.linear1(x_negative)
        x_negative = torch.nn.functional.relu(x_negative)
        x_negative = self.linear2(x_negative)
        x_negative = torch.sigmoid(x_negative)
        return x_positive.squeeze(), x_negative.squeeze()

In [ ]:
# Split data to test and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
train_size = round(0.8*data.edge_index.shape[1])
model = GCN(planetoid).to(device)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
# train the model
for epoch in range(200):
    optimizer.zero_grad()
    target = torch.concat((torch.ones((train_size,), device = device), torch.zeros((train_size,), device = device)))

    out = model(data.x, 
                data.edge_index[:, :train_size],
                data.edge_index[:, :train_size])
    pred = torch.concat(out)
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()

In [ ]:
# use recall as evaluation metrics
pred = model(data.x, 
             data.edge_index[:, :train_size],
             data.edge_index[:, train_size:])[0]
correct = (pred >= 0.5).sum()
recall = int(correct) / (data.edge_index.shape[1] - train_size)
print(f'Recall: {recall:.4f}')

Recall: 0.9061


In [ ]:
# GCN for GraphSage
class GraphSage(torch.nn.Module):
    def __init__(self, dataset):
        super().__init__()
        self.conv1 = torch_geometric.nn.SAGEConv(dataset.num_node_features, 16)
        self.conv2 = torch_geometric.nn.SAGEConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.nn.functional.log_softmax(x, dim=1)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = GraphSage(planetoid).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = torch.nn.functional.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7690


In [ ]:
# GrapgSage for linke prediction
class GraphSage(torch.nn.Module):
    def __init__(self, dataset):
        super().__init__()
        self.conv1 = torch_geometric.nn.SAGEConv(dataset.num_node_features, 16)
        self.conv2 = torch_geometric.nn.SAGEConv(16, 12)
        self.linear1 = torch.nn.Linear(24, 24)
        self.linear2 = torch.nn.Linear(24, 1)

    def forward(self, x, train_edge_index, pred_edge_index):
        x = self.conv1(x, train_edge_index)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x, train_edge_index)
        x = torch.nn.functional.relu(x)

        x_positive = torch.cat((x[pred_edge_index[0,:],:], 
                                x[pred_edge_index[1,:],:]), dim=1)
        x_positive = self.linear1(x_positive)
        x_positive = torch.nn.functional.relu(x_positive)
        x_positive = self.linear2(x_positive)
        x_positive = torch.sigmoid(x_positive)

        x_negative = torch.cat((x[torch.randint(high = x.shape[0], size = (pred_edge_index.shape[1],)),:],
                                x[pred_edge_index[1,:],:]), dim=1)
        x_negative = self.linear1(x_negative)
        x_negative = torch.nn.functional.relu(x_negative)
        x_negative = self.linear2(x_negative)
        x_negative = torch.sigmoid(x_negative)
        return x_positive.squeeze(), x_negative.squeeze()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
train_size = round(0.8*data.edge_index.shape[1])
model = GraphSage(planetoid).to(device)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    optimizer.zero_grad()
    target = torch.concat((torch.ones((train_size,), device = device), torch.zeros((train_size,), device = device)))

    out = model(data.x, 
                data.edge_index[:, :train_size],
                data.edge_index[:, :train_size])
    pred = torch.concat(out)
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()

pred = model(data.x, 
             data.edge_index[:, :train_size],
             data.edge_index[:, train_size:])[0]
correct = (pred >= 0.5).sum()
recall = int(correct) / (data.edge_index.shape[1] - train_size)
print(f'Recall: {recall:.4f}')

Recall: 0.7655
